# Github repo

clone github repository

In [1]:
!git clone https: // github.com/thomasmaliappis/context-group-detection.git

Cloning into 'context-group-detection'...
remote: Enumerating objects: 805, done.
remote: Counting objects: 100% (117/117), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 805 (delta 72), reused 78 (delta 36), pack-reused 688
Receiving objects: 100% (805/805), 1.54 MiB | 9.25 MiB/s, done.
Resolving deltas: 100% (459/459), done.


move in repository folder

In [2]:
%cd context-group-detection

/content/context-group-detection


change branch

In [3]:
!git checkout dante

Branch 'dante' set up to track remote branch 'dante' from 'origin'.
Switched to a new branch 'dante'


# Drive

mount drive

In [4]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


# Requirements
install missing packages

In [5]:
!pip install xlsxwriter

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.0/153.0 kB 6.9 MB/s eta 0:00:00


In [6]:
!pip install pykalman

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.9/228.9 kB 10.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pykalman: filename=pykalman-0.9.5-py3-none-any.whl size=48442 sha256=6e4bc2be82bb0f245bfba9d1e4997dc2cd777cfecb3c3efc4580756ca7118013
  Stored in directory: /root/.cache/pip/wheels/76/33/ef/5f332226e13a5089c6dd4b01cc2bcb59491d18f955fa2d3807
Successfully built pykalman


# Imports

In [7]:
import numpy as np
import tensorflow as tf
from keras.callbacks import EarlyStopping, TensorBoard
from keras.layers import Dense, Conv1D, LSTM, concatenate, Input, Flatten, Dropout, BatchNormalization
from keras.models import Model
from keras.optimizers import Adam
from keras.regularizers import l2

from models.utils import ValLoss, load_data, save_model_data

# Functions

create model

In [8]:
def build_model(context_size, consecutive_frames, features, reg_amount, drop_amount, learning_rate, lstm_units=64,
                pair_filters=[32], context_filters=[32], combination_filters=[64]):
    """
    Builds model based on given parameters.
    :param context_size: size of context
    :param consecutive_frames: number of frames per scene
    :param features: features
    :param reg_amount: regularization factor
    :param drop_amount: dropout rate
    :param learning_rate: learning rate
    :param lstm_units: units to be used in lstm layers
    :param pair_filters: filters to be used in conv1d layers for pair
    :param context_filters: filters to be used in conv1d layers for context
    :param combination_filters: units to be used in dense layer
    :return: model
    """
    inputs = []

    # pair branch
    # create input layers
    pair_inputs = []
    for i in range(2):
        pair_input = Input(shape=(consecutive_frames, features), name='pair_{}'.format(i))
        pair_inputs.append(pair_input)
        inputs.append(pair_input)

    pair_layers = []
    for pair_input in pair_inputs:
        lstm = LSTM(lstm_units, return_sequences=True)(pair_input)
        pair_layers.append(lstm)

    reg = l2(reg_amount)

    pair_concatenated = concatenate(pair_layers)

    pair_x = pair_concatenated
    for filters in pair_filters:
        pair_x = Conv1D(filters=filters, kernel_size=3, kernel_regularizer=reg, activation='relu',
                        name='pair_conv_{}'.format(filters))(pair_x)
        pair_x = Dropout(drop_amount)(pair_x)
        pair_x = BatchNormalization()(pair_x)
    pair_conv = pair_x
    # max_pool = MaxPooling1D()(batch_norm)
    # drop = Dropout(drop_amount)(max_pool)
    # batch_norm = BatchNormalization()(drop)
    pair_layer = pair_conv

    # context branch
    context_inputs = []
    for i in range(context_size):
        context_input = Input(shape=(consecutive_frames, features), name='context_{}'.format(i))
        context_inputs.append(context_input)
        inputs.append(context_input)

    context_layers = []
    for context_input in context_inputs:
        lstm = LSTM(lstm_units, return_sequences=True)(context_input)
        context_layers.append(lstm)

    context_concatenated = concatenate(context_layers)

    context_x = context_concatenated
    for filters in context_filters:
        context_x = Conv1D(filters=filters, kernel_size=3, kernel_regularizer=reg, activation='relu',
                           name='context_conv_{}'.format(filters))(context_x)
        context_x = Dropout(drop_amount)(context_x)
        context_x = BatchNormalization()(context_x)
    context_conv = context_x
    # max_pool = MaxPooling1D()(batch_norm)
    # drop = Dropout(drop_amount)(max_pool)
    # batch_norm = BatchNormalization()(drop)
    context_layer = context_conv

    # Concatenate the outputs of the two branches
    combined = concatenate([pair_layer, context_layer], axis=1)
    flatten = Flatten()(combined)

    combination_x = flatten
    for filters in combination_filters:
        combination_x = Dense(units=filters, use_bias='True', kernel_regularizer=reg, activation='relu',
                              kernel_initializer="he_normal")(combination_x)
        combination_x = Dropout(drop_amount)(combination_x)
        combination_x = BatchNormalization()(combination_x)

    # Output layer
    output = Dense(1, activation='sigmoid')(combination_x)

    # Create the model with two inputs and one output
    model = Model(inputs=[inputs], outputs=output)

    # Compile the model
    opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, decay=1e-5, amsgrad=False, clipvalue=0.5)
    model.compile(optimizer=opt, loss="binary_crossentropy", metrics=['mse'])

    return model

class to handle arguments instead of argument parser

In [10]:
class Args:
    def __init__(self, dataset, dataset_path, epochs, features, agents, frames, batch_size, reg, dropout, learning_rate,
                 gmitre_calc):
        self.dataset = dataset
        self.dataset_path = dataset_path
        self.epochs = epochs
        self.features = features
        self.agents = agents
        self.frames = frames
        self.batch_size = batch_size
        self.reg = reg
        self.dropout = dropout
        self.learning_rate = learning_rate
        self.gmitre_calc = gmitre_calc


# Main

set argument constants

In [11]:
args = Args(
    dataset='eth',
    dataset_path='./datasets/ETH/seq_eth',
    epochs=50,
    features=4,
    agents=10,
    frames=10,
    batch_size=1024,
    reg=0.0000001,
    dropout=0.35,
    learning_rate=0.0001,
    gmitre_calc=True
)

initialise variables

In [12]:
tf.random.set_seed(0)
np.random.seed(0)

load data

In [13]:
train, test, val = load_data('/content/drive/My Drive/datasets/{}_{}_{}'.format(args.dataset, args.frames, args.agents))

create and train model

In [14]:
model = build_model(args.agents - 2, args.frames, args.features, args.reg, args.dropout, args.learning_rate,
                    # pair_filters=[32, 128, 256], context_filters=[64, 128, 256], combination_filters=[256, 64]
                    )

tensorboard = TensorBoard(log_dir='./logs')
early_stop = EarlyStopping(monitor='val_loss', patience=10)
history = ValLoss(val, args.dataset, args.dataset_path, True, args.gmitre_calc)

model.fit(train[0], train[1], epochs=args.epochs, batch_size=args.batch_size,
          validation_data=(val[0], val[1]), callbacks=[tensorboard, early_stop, history])

Epoch 1/50
 5/85 [>.............................] - ETA: 2s - loss: 0.8493 - mse: 0.3025

85/85 [==============================] - 50s 316ms/step - loss: 0.6358 - mse: 0.2187 - val_loss: 0.6658 - val_mse: 0.2365
Epoch 2/50
85/85 [==============================] - 209s 2s/step - loss: 0.3964 - mse: 0.1257 - val_loss: 0.6155 - val_mse: 0.2115
Epoch 3/50
85/85 [==============================] - 30s 356ms/step - loss: 0.2843 - mse: 0.0846 - val_loss: 0.5499 - val_mse: 0.1801
Epoch 4/50
85/85 [==============================] - 14s 163ms/step - loss: 0.2159 - mse: 0.0610 - val_loss: 0.4618 - val_mse: 0.1408
Epoch 5/50
85/85 [==============================] - 10s 120ms/step - loss: 0.1666 - mse: 0.0451 - val_loss: 0.3498 - val_mse: 0.0962
Epoch 6/50
85/85 [==============================] - 13s 158ms/step - loss: 0.1303 - mse: 0.0340 - val_loss: 0.2399 - val_mse: 0.0575
Epoch 7/50
85/85 [==============================] - 10s 114ms/step - loss: 0.1050 - mse: 0.0270 - val_loss: 0.1500 - val_mse: 0.0327
Epoch 8/50
85/85 [==============================] - 8s 98ms/step - loss: 0.0880 - 

keep track of evaluation metrics

In [15]:
save_model_data(args.dataset, args.reg, args.dropout, history, test, True, gmitre_calc=args.gmitre_calc)

saving model to models/eth/pair_predictions_1
1913/1913 [==============================] - 16s 8ms/step
saved best val model as /best_val_model.h5
